In [2]:
%load_ext Cython

In [9]:
%%cython
from cpython cimport array as c_array
from cpython.array cimport array as carray
from array import array

cdef struct myStruct:
    float a
    int b
    int* c


cdef void sub(int[:] arg):
    arg[0] = 12
    
def test():
    cdef c_array.array a = array('i', [0,1,2])
    cdef c_array.array b = array('i', [3,4,5])
    c_array.extend(a,b)
    c_array.resize(a,len(a)+1)
    cdef int[:] ca = a
    ca[0] = 5
    ca[-1] = 10
    print(ca[0])
    print(a[0])
    print(len(a))
    print(len(ca))
    print(a)
    sub(a)
    print(a)

    
cdef test2(int arg1, arg2):
    cdef myStruct s
    s.a = 1.
    s.b = 2
    cdef c_array.array arr = array('i',[0,1,2])
    s.c = arr.data.as_ints
    print(s.a, s.b, s.c[0])
    return ("hello", "goodbye")


cdef class myClass:
    cdef c_array.array a
    def __init__(self, a):
        self.a = a

def test3():
    cdef c_array.array arr = array('f',[1,2,3])
    arr.append(1.)
    print(arr)
    cdef myClass mc = myClass(arr)
    print(len(mc.a))

cdef test4_aux(c_array.array arg):
    arg.append(5)
    
def test4():
    cdef c_array.array arr = array('i',[])
    test4_aux(arr)
    print(arr)

    
def test_extend_1():
    cdef c_array.array arr = array('i',[])
    for i in range(10000):
        arr.append(i)
    
def test_extend_2():
    cdef c_array.array arr = array('i',[])
    for i in range(10000):
        c_array.resize(arr,i+1)
        arr[i] = i

def test_extend_3():
    cdef c_array.array arr = array('i',[])
    c_array.resize(arr, 10000)
    for i in range(10000):
        arr[i] = i

def test_extend_4():
    cdef c_array.array arr = array('i',[0])*10000
    for i in range(10000):
        arr[i] = i

def test_extend_5():
    cdef c_array.array arr = array('i',[0])
    cdef int[:] c_arr = arr
    cdef n = 1
    cdef N = 1
    for i in range(10000):
        if n == N:
            N = N*2
            c_array.resize(arr, N)
            c_arr = arr
        n += 1
        c_arr[i] = i
    c_array.resize(arr, n)

def test_extend_6():
    cdef list arr = []
    for i in range(10000):
        arr.append(i)
from collections import deque
def test_6():
    d = deque(maxlen=5)
    for i in range(10):
        d.append(i)
    print(d)
    
def test_py():
    print(test2(5, ("hey", "buddy")))

In [6]:
#test()
#print("\n\n")
#test2()
#test_py()
#test4()
#%timeit test_extend_1()
#%timeit test_extend_2()
#%timeit test_extend_3()
#%timeit test_extend_4()
#%timeit test_extend_5()
%timeit test_extend_6()
#test_extend_5()

TypeError: 'list' does not support the buffer interface